# O/D Efficiency

Efficiency is calculated by +1 for being on the line when we score and -1 for being on the line when we get scored on, then normalized.

Use 'Points vs. ...' csv for what happened on each point and 'Player Stats vs. ...' for who was on the line for that point

In [92]:
%pylab inline
import pandas as pd

Populating the interactive namespace from numpy and matplotlib


In [93]:
player_stats_df = pd.read_csv('WY-Wolves/Player Stats vs. CI-Vapour 2017-03-29_16-30.csv')
points_df = pd.read_csv('WY-Wolves/Points vs. CI-Vapour 2017-03-29_16-30.csv')

In [94]:
# get the points each player played in

players = player_stats_df.loc[:, ['Player', 'Points played']]
print(players)

              Player                   Points played
0      #1 Joyce Wang  1,2,3,4,5,6,7,9,10,12,13,15,16
1     #14 Cecilia Wu     1,2,3,4,5,6,8,9,11,12,14,15
2   #27 Daniel Zhang         1,2,3,4,6,8,10,12,14,16
3       #28 Tina Zhu          2,5,7,8,10,11,13,14,16
4   #22 Paul Simpson           1,2,3,6,8,10,12,14,16
5    #69 Justin Chen              1,4,5,7,9,11,13,15
6      #55 Tigger Li              1,2,5,7,9,11,13,15
7    #52 Emily Zhang                1,4,7,9,11,13,15
8       #0 Julie Luo               3,6,8,10,12,14,16
9       #76 Jack Xia                  3,4,7,10,13,16
10         Alex Chen                  3,4,7,10,13,16
11      #19 Ben Wang                     2,6,9,12,15
12      #41 Billy Ni                       5,8,11,14
13    #36 Jason Ling                       6,9,12,15
14    #66 Jerry Chen                       5,8,11,14


In [95]:
# find the o_points, the ones we scored on and the ones we got scored on

o_points = points_df[points_df['Started on offense?'] == 1]
o_points_won = o_points[o_points['Scored?'] == 1]
o_points_lost = o_points[o_points['Scored?'] == 0]

In [96]:
# dictionary with each o point that we scored
# keys are the point numbers
# values are lists of players on the line that point

points_won = o_points_won.loc[:, ['Point']].values

o_lines_plus = {}

if points_won.size > 0:
    for point in np.nditer(points_won):
        point = int(point)
        for row in players.iterrows():
            player_points = row[1][1].split(',')
            if str(point) in player_points:
                if point in o_lines_plus.keys():
                    o_lines_plus[point].append(row[1][0])
                else:
                    o_lines_plus[point] = [row[1][0]]

for point, line in o_lines_plus.items():
    print('Point',point)
    for player in line:
        print('\t', player)
else:
    print({})

Point 1
	 #1 Joyce Wang
	 #14 Cecilia Wu
	 #27 Daniel Zhang
	 #22 Paul Simpson
	 #69 Justin Chen
	 #55 Tigger Li
	 #52 Emily Zhang
Point 5
	 #1 Joyce Wang
	 #14 Cecilia Wu
	 #28 Tina Zhu
	 #69 Justin Chen
	 #55 Tigger Li
	 #41 Billy Ni
	 #66 Jerry Chen
Point 12
	 #1 Joyce Wang
	 #14 Cecilia Wu
	 #27 Daniel Zhang
	 #22 Paul Simpson
	 #0 Julie Luo
	 #19 Ben Wang
	 #36 Jason Ling
Point 14
	 #14 Cecilia Wu
	 #27 Daniel Zhang
	 #28 Tina Zhu
	 #22 Paul Simpson
	 #0 Julie Luo
	 #41 Billy Ni
	 #66 Jerry Chen
{}


In [97]:
# dictionary with each o point that we got scored on
# keys are the point numbers
# values are lists of players on the line that point

points_lost = o_points_lost.loc[:,['Point']].values

o_lines_minus = {}

if points_lost.size > 0:
    for point in np.nditer(points_lost):
        point = int(point)
        for row in players.iterrows():
            player_points = row[1][1].split(',')
            if str(point) in player_points:
                if point in o_lines_minus.keys():
                    o_lines_minus[point].append(row[1][0])
                else:
                    o_lines_minus[point] = [row[1][0]]
                    

for point, line in o_lines_minus.items():
    print('Point',point)
    for player in line:
        print('\t', player)
else:
    print({})

{}


In [99]:
# get the number of o points played for each player as a dictionary
# keys are player names
# values are number of o points played

points_played = o_points.loc[:,['Point']].values
o_points_played = {}
if points_played.size > 0:
    for point in np.nditer(points_played):
        point = int(point)
        for row in players.iterrows():
            player_points = row[1][1].split(',')
            if str(point) in player_points:
                if row[1][0] in o_points_played.keys():
                    o_points_played[row[1][0]] += 1
                else:
                    o_points_played[row[1][0]] = 1
                    
for player, points in o_points_played.items():
    print(player, ':\t', points)

#1 Joyce Wang :	 3
#14 Cecilia Wu :	 4
#27 Daniel Zhang :	 3
#22 Paul Simpson :	 3
#69 Justin Chen :	 2
#55 Tigger Li :	 2
#52 Emily Zhang :	 1
#28 Tina Zhu :	 2
#41 Billy Ni :	 2
#66 Jerry Chen :	 2
#0 Julie Luo :	 2
#19 Ben Wang :	 1
#36 Jason Ling :	 1


In [100]:
# calculate offensive efficiency as a dictionary
# keys are player names
# values are their effiencies between -1 and 1

# create the dictionary and add all the positives
o_eff = {}
for point, line in o_lines_plus.items():
    for player in line:
        if player in o_eff.keys():
            o_eff[player] += 1
        else:
            o_eff[player] = 1

# add all the minuses
for point, line in o_lines_minus.items():
    for player in line:
        if player in o_eff.keys():
            o_eff[player] -= 1
        else:
            o_eff[player] = -1

# divide by the number of o points played
for player, eff in o_eff.items():
    o_eff[player] = eff/o_points_played[player]
    
for player, eff in o_eff.items():
    print(player, ':\t', eff)

#1 Joyce Wang :	 1.0
#14 Cecilia Wu :	 1.0
#27 Daniel Zhang :	 1.0
#22 Paul Simpson :	 1.0
#69 Justin Chen :	 1.0
#55 Tigger Li :	 1.0
#52 Emily Zhang :	 1.0
#28 Tina Zhu :	 1.0
#41 Billy Ni :	 1.0
#66 Jerry Chen :	 1.0
#0 Julie Luo :	 1.0
#19 Ben Wang :	 1.0
#36 Jason Ling :	 1.0


In [101]:
# find the d_points, the ones we scored on and the ones we got scored on

d_points = points_df[points_df['Started on offense?'] == 0]
d_points_won = d_points[d_points['Scored?'] == 1]
d_points_lost = d_points[d_points['Scored?'] == 0]

In [102]:
# dictionary with each d point that we scored
# keys are the point numbers
# values are lists of players on the line that point

points_won = d_points_won.loc[:, ['Point']].values

d_lines_plus = {}

if points_won.size > 0:
    for point in np.nditer(points_won):
        point = int(point)
        for row in players.iterrows():
            player_points = row[1][1].split(',')
            if str(point) in player_points:
                if point in d_lines_plus.keys():
                    d_lines_plus[point].append(row[1][0])
                else:
                    d_lines_plus[point] = [row[1][0]]

for point, line in d_lines_plus.items():
    print('Point',point)
    for player in line:
        print('\t', player)
else:
    print({})

Point 2
	 #1 Joyce Wang
	 #14 Cecilia Wu
	 #27 Daniel Zhang
	 #28 Tina Zhu
	 #22 Paul Simpson
	 #55 Tigger Li
	 #19 Ben Wang
Point 3
	 #1 Joyce Wang
	 #14 Cecilia Wu
	 #27 Daniel Zhang
	 #22 Paul Simpson
	 #0 Julie Luo
	 #76 Jack Xia
	 Alex Chen
Point 6
	 #1 Joyce Wang
	 #14 Cecilia Wu
	 #27 Daniel Zhang
	 #22 Paul Simpson
	 #0 Julie Luo
	 #19 Ben Wang
	 #36 Jason Ling
Point 7
	 #1 Joyce Wang
	 #28 Tina Zhu
	 #69 Justin Chen
	 #55 Tigger Li
	 #52 Emily Zhang
	 #76 Jack Xia
	 Alex Chen
Point 8
	 #14 Cecilia Wu
	 #27 Daniel Zhang
	 #28 Tina Zhu
	 #22 Paul Simpson
	 #0 Julie Luo
	 #41 Billy Ni
	 #66 Jerry Chen
Point 9
	 #1 Joyce Wang
	 #14 Cecilia Wu
	 #69 Justin Chen
	 #55 Tigger Li
	 #52 Emily Zhang
	 #19 Ben Wang
	 #36 Jason Ling
Point 10
	 #1 Joyce Wang
	 #27 Daniel Zhang
	 #28 Tina Zhu
	 #22 Paul Simpson
	 #0 Julie Luo
	 #76 Jack Xia
	 Alex Chen
Point 15
	 #1 Joyce Wang
	 #14 Cecilia Wu
	 #69 Justin Chen
	 #55 Tigger Li
	 #52 Emily Zhang
	 #19 Ben Wang
	 #36 Jason Ling
Point 16
	 #1 

In [103]:
# dictionary with each d point that we got scored on
# keys are the point numbers
# values are lists of players on the line that point

points_lost = d_points_lost.loc[:,['Point']].values

d_lines_minus = {}

if points_lost.size > 0:
    for point in np.nditer(points_lost):
        point = int(point)
        for row in players.iterrows():
            player_points = row[1][1].split(',')
            if str(point) in player_points:
                if point in d_lines_minus.keys():
                    d_lines_minus[point].append(row[1][0])
                else:
                    d_lines_minus[point] = [row[1][0]]
                    

for point, line in d_lines_minus.items():
    print('Point',point)
    for player in line:
        print('\t', player)
else:
    print({})

Point 4
	 #1 Joyce Wang
	 #14 Cecilia Wu
	 #27 Daniel Zhang
	 #69 Justin Chen
	 #52 Emily Zhang
	 #76 Jack Xia
	 Alex Chen
Point 11
	 #14 Cecilia Wu
	 #28 Tina Zhu
	 #69 Justin Chen
	 #55 Tigger Li
	 #52 Emily Zhang
	 #41 Billy Ni
	 #66 Jerry Chen
Point 13
	 #1 Joyce Wang
	 #28 Tina Zhu
	 #69 Justin Chen
	 #55 Tigger Li
	 #52 Emily Zhang
	 #76 Jack Xia
	 Alex Chen
{}


In [104]:
# get the number of d points played for each player as a dictionary
# keys are player names
# values are number of o points played

points_played = d_points.loc[:,['Point']].values
d_points_played = {}
if points_played.size > 0:
    for point in np.nditer(points_played):
        point = int(point)
        for row in players.iterrows():
            player_points = row[1][1].split(',')
            if str(point) in player_points:
                if row[1][0] in d_points_played.keys():
                    d_points_played[row[1][0]] += 1
                else:
                    d_points_played[row[1][0]] = 1
                    
for player, points in d_points_played.items():
    print(player, ':\t', points)

#1 Joyce Wang :	 10
#14 Cecilia Wu :	 8
#27 Daniel Zhang :	 7
#28 Tina Zhu :	 7
#22 Paul Simpson :	 6
#55 Tigger Li :	 6
#19 Ben Wang :	 4
#0 Julie Luo :	 5
#76 Jack Xia :	 6
Alex Chen :	 6
#69 Justin Chen :	 6
#52 Emily Zhang :	 6
#36 Jason Ling :	 3
#41 Billy Ni :	 2
#66 Jerry Chen :	 2


In [105]:
# calculate defensive efficiency as a dictionary
# keys are player names
# values are their effiencies between -1 and 1

# create the dictionary and add all the positives

d_eff = {}
for point, line in d_lines_plus.items():
    for player in line:
        if player in d_eff.keys():
            d_eff[player] += 1
        else:
            d_eff[player] = 1

# subtract all the negatives
for point, line in d_lines_minus.items():
    for player in line:
        if player in d_eff.keys():
            d_eff[player] -= 1
        else:
            d_eff[player] = -1

# divide by the number of points played
for player, eff in d_eff.items():
    d_eff[player] = eff/d_points_played[player]
    
for player, eff in d_eff.items():
    print(player, ':\t', eff)

#1 Joyce Wang :	 0.6
#14 Cecilia Wu :	 0.5
#27 Daniel Zhang :	 0.7142857142857143
#28 Tina Zhu :	 0.42857142857142855
#22 Paul Simpson :	 1.0
#55 Tigger Li :	 0.3333333333333333
#19 Ben Wang :	 1.0
#0 Julie Luo :	 1.0
#76 Jack Xia :	 0.3333333333333333
Alex Chen :	 0.3333333333333333
#36 Jason Ling :	 1.0
#69 Justin Chen :	 0.0
#52 Emily Zhang :	 0.0
#41 Billy Ni :	 0.0
#66 Jerry Chen :	 0.0
